In [ ]:
%%capture cap --no-stderr

with open('train.txt', 'w') as f:
    f.write(cap.stdout)

del cap 

import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

components = [ 10, 25, 50, 100, 784 ]
y = pickle.load(open('target.pkl', 'r'))

for c in components:
    
    print '-------------------------------------------------------'
    print
    print 'Training and optimisation report with {0} principal component'.format(c)
    print
    filename = 'data-{0}-components.pkl'.format(c)
    X = pickle.load(open(filename, 'r'))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1/7), random_state=0)
    
    # Save split test train datasets with pickle    
    X_train_file = 'X_train-for-{0}-components.pkl'.format(c)
    file = open(X_train_file, 'w')
    pickle.dump(X_train, file)
    file.close()

    X_test_file = 'X_test-for-{0}-components.pkl'.format(c)
    file = open(X_test_file, 'w')
    pickle.dump(X_test, file)
    file.close()

    y_train_file = 'y_train-for-{0}-components.pkl'.format(c)
    file = open(y_train_file, 'w')
    pickle.dump(y_train, file)
    file.close()

    y_test_file = 'y_test-for-{0}-components.pkl'.format(c)
    file = open(y_test_file, 'w')
    pickle.dump(y_test, file)
    file.close()
    
    tuned_parameters = [
                        {
                        'kernel': ['rbf'],
                        'gamma': [1e-3, 1e-4],
                        'C': [1, 10, 100, 1000]
                        },
                        {
                        'kernel': ['poly'],
                        'gamma': [1e-3, 1e-4],
                        'C': [1, 10, 100, 1000],
                        'degree': [5, 6, 7]
                        }
                        ]

    print 'Grid Search'
    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring='accuracy')
    clf.fit(X_train, y_train)
    
    print 'Best parameters set found on development set:'
    print clf.best_params_
    print
    print 'Grid scores on development set:'
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print '%0.3f (+/-%0.03f) for %r' % (mean, std * 2, params)
    print
    print
    print '-------------------------------------------------------'
    print
    
    clf_file = 'model-for-{0}-components.pkl'.format(c)
    file = open(clf_file, 'w')
    pickle.dump(clf, file)
    file.close()
 

70000
Grid Search
